In [ ]:
import pandas as pd
from analysis import get_market_participants
from utils import get_market_df, TRADERS_IN_FINANCIAL_FUTURES_FUT, LEGUACY_FUT, DISAGGREGATED_FUT

In [ ]:
df = get_market_df(cot_report_type=LEGUACY_FUT)

In [ ]:
import json

participants = get_market_participants(df)

print(json.dumps(participants, indent=4))

In [ ]:
picked_participant = participants['noncommercial']
print(json.dumps(picked_participant, indent=4))

In [ ]:
picked_participant_df = pd.DataFrame({})

for side in ['long', 'short',  'spread']:
  if side not in picked_participant: continue
  picked_participant_df[side] = df[picked_participant[side]]
  picked_participant_df[f"{side}_change"] = pd.to_numeric(df[picked_participant[f'{side}_change']], errors="coerce")
  picked_participant_df[f"{side}_oi"] = df[picked_participant[f"{side}_oi"]]

picked_participant_df['Net'] = picked_participant_df['long'] - picked_participant_df['short']
picked_participant_df['Net % change'] = 100 * picked_participant_df['Net'].pct_change(periods=1)

picked_participant_df['long %'] = 100 * picked_participant_df['long'] / (
    picked_participant_df['long'] +
    picked_participant_df['short'] +
    (picked_participant_df['spread'] if 'spread' in picked_participant_df else 0)
)

picked_participant_df['short %'] = 100 * picked_participant_df['short'] / (
    picked_participant_df['long'] +
    picked_participant_df['short'] +
    (picked_participant_df['spread'] if 'spread' in picked_participant_df else 0)
)

picked_participant_df = picked_participant_df.tail(23)

picked_participant_df

In [ ]:
color_map = {
    "long": "Greens",
    "short": "YlOrRd",
    "spread": "seismic"
}

styled = picked_participant_df.style

for side in ["long", "short", "spread"]:

    if side not in picked_participant: continue
    (
      styled.background_gradient(cmap=color_map[side], subset=[side], vmin=picked_participant_df[side].min(), vmax=picked_participant_df[side].max())
            .background_gradient(cmap=color_map[side], subset=[f'{side}_change'], vmin=picked_participant_df[f'{side}_change'].min(), vmax=picked_participant_df[f'{side}_change'].max())
            .background_gradient(cmap=color_map[side], subset=[f'{side}_oi'], vmin=picked_participant_df[f'{side}_oi'].min(), vmax=picked_participant_df[f'{side}_oi'].max())
    )

(
    styled.background_gradient(cmap="Blues", subset=["long %"], vmin=picked_participant_df["long %"].min(), vmax=picked_participant_df["long %"].max())
          .background_gradient(cmap="Purples", subset=["short %"], vmin=picked_participant_df["short %"].min(), vmax=picked_participant_df["short %"].max())
          .background_gradient(cmap="Wistia", subset=['Net'], vmin=picked_participant_df['Net'].min(), vmax=picked_participant_df['Net'].max())
          .background_gradient(cmap="Wistia", subset=['Net % change'], vmin=picked_participant_df['Net % change'].min(), vmax=picked_participant_df['Net % change'].max())
)

styled.format("{:g}")
styled.format_index("{:%Y-%m-%d}", axis=0)

styled